In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from k_diffusion.callback import SampleCallback, load_model
from k_diffusion.config import SampleCallbackConfig
from pathlib import Path

model_id = "htkunb2s"
model_step = -1

model_path = Path("/shared/amyxlu/kdplaid/checkpoints") / model_id 
model, inner_model, model_config = load_model(model_path, model_step)

#         ckpt = torch.load(ckpt_path, map_location="cpu")

#         # load the previous config for consistency.
#         # if a JSON config is saved, use it -- allows us to hack configs but use past checkpoints
#         if (checkpoints_dir / "config.json").exists():
#             argsdict = json.load(open(checkpoints_dir / "config.json"))
#             args = K.config.dataclass_from_dict(K.config.TrainArgs, argsdict) 

# cfg = SampleCallbackConfig()
# sampler = SampleCallback(cfg)
